In [1]:
import pandas as pd
import yaml
import numpy as np
import os
import datetime

In [2]:
columns = ['match_date','match_type','teams','venue','overs','winner'
           ,'innings_number','team','ball','batsman','non_striker'
           ,'runs_batsman','runs_extras','runs_total'
           ,'wicket_kind','wicket_player_out','wicket_fielders']

In [3]:
def data_prep(name):
    file = '/Users/z001t72/Documents/Training/cricket/data/all_male/' + name
    with open(file, 'r') as f:
        df = pd.io.json.json_normalize(yaml.load(f))
    
    if df['info.match_type'][0] in ['T20','ODI'] and 'info.outcome.winner' in df.columns:
        df1 = df['innings'].apply(pd.Series).stack()
        df1 = df1.reset_index()
        df1 = df1[df1['level_1'] <= 1] #limiting only to 2 innings. this will exclude innings like super over
        df1.columns = ['nr1','nr2','innings']
        df1 = df1[['innings']]
        df1 = pd.DataFrame(df1.innings.values.tolist())
        df1.columns = ['innings_1','innings_2']
        df1['innings'] = df1.innings_1.combine_first(df1.innings_2)
        df1['innings_number'] = np.where(df1['innings_1'].isna(), '2', '1')
        df1 = df1[['innings','innings_number']]
        df1[['deliveries','team']] = pd.DataFrame(df1.innings.values.tolist(), index = df1.index)[['deliveries','team']] # sometimes there are additional columns that appear (like absent hurt), hence only selecting deliveries & team
        df1 = df1.set_index(['innings_number', 'team'])['deliveries'].apply(pd.Series).stack()
        df1 = df1.reset_index()
        df1.columns = ['innings_number','team','notreqd','balls']

        df1['ball'] = df1['balls'].apply(lambda x: list(x.keys())[0])
        df1['batsman'] = df1['balls'].apply(lambda x: list(x.values())[0].get('batsman'))
        df1['non_striker'] = df1['balls'].apply(lambda x: list(x.values())[0].get('non_striker'))
        df1['runs_batsman'] = df1['balls'].apply(lambda x: list(x.values())[0].get('runs', {}).get('batsman'))
        df1['runs_extras'] = df1['balls'].apply(lambda x: list(x.values())[0].get('runs', {}).get('extras'))
        df1['runs_total'] = df1['balls'].apply(lambda x: list(x.values())[0].get('runs', {}).get('total'))
        df1['wicket_kind'] = df1['balls'].apply(lambda x: list(x.values())[0].get('wicket', {}).get('kind'))
        df1['wicket_player_out'] = df1['balls'].apply(lambda x: list(x.values())[0].get('wicket', {}).get('player_out'))
        df1['wicket_fielders'] = df1['balls'].apply(lambda x: list(x.values())[0].get('wicket', {}).get('fielders'))

        # some dates are available as string and some as datetime
        if type(df['info.dates'][0][0]) is str: df1['match_date'] = str(df['info.dates'][0][0])
        elif type(df['info.dates'][0][0]) is datetime.date: df1['match_date'] = "{:%Y-%m-%d}".format(df['info.dates'][0][0])
        else: df1['match_date'] = 'NA'
            
        df1['match_type'] = df['info.match_type'][0]
        df1['teams'] = str(df['info.teams'][0])
        df1['venue'] = df['info.venue'][0]
#         df1['city'] = df["info.city"][0]
        df1['overs'] = df['info.overs'][0]
        df1['winner'] = df['info.outcome.winner'][0]

        df1 = df1[columns]
        return(df1)
    
    elif 'result' in df.columns and df['info.outcome.result'][0] == 'no result':
        print('...skipping no-result contests...')
        
    elif df['info.match_type'][0] in ['Test']:
        print('...skipping Test matches...')
    
    else:
        print('...skipping file, reason unknown...')

In [4]:
master_odi_t20 = pd.DataFrame(columns = columns)

In [ ]:
directory = '/Users/z001t72/Documents/Training/cricket/data/all_male/'

In [ ]:
i = 0
for filename in os.listdir(directory):
    if filename.endswith(".yaml"):
        i = i + 1
        print('processing file number: ',i,' filename: ',filename)
        match_data = data_prep(filename)
        master_odi_t20 = master_odi_t20.append(match_data)
        if i%100 == 0:
            master_odi_t20.to_csv("master_odi_t20.csv")
    else:
        continue

processing file number:  1  filename:  598067.yaml
processing file number:  2  filename:  401072.yaml
...skipping Test matches...
processing file number:  3  filename:  582190.yaml
processing file number:  4  filename:  897721.yaml
processing file number:  5  filename:  829725.yaml
processing file number:  6  filename:  1029827.yaml
...skipping Test matches...
processing file number:  7  filename:  501255.yaml
processing file number:  8  filename:  267712.yaml
processing file number:  9  filename:  1082637.yaml
processing file number:  10  filename:  474467.yaml
processing file number:  11  filename:  1114884.yaml
processing file number:  12  filename:  1167153.yaml
processing file number:  13  filename:  1128841.yaml
...skipping file, reason unknown...
processing file number:  14  filename:  778041.yaml
...skipping file, reason unknown...
processing file number:  15  filename:  289108.yaml
processing file number:  16  filename:  419157.yaml
processing file number:  17  filename:  5658

processing file number:  139  filename:  287878.yaml
processing file number:  140  filename:  355995.yaml
processing file number:  141  filename:  810833.yaml
processing file number:  142  filename:  947223.yaml
...skipping file, reason unknown...
processing file number:  143  filename:  1136561.yaml
processing file number:  144  filename:  749771.yaml
processing file number:  145  filename:  914211.yaml
processing file number:  146  filename:  501238.yaml
processing file number:  147  filename:  766931.yaml
...skipping Test matches...
processing file number:  148  filename:  1022355.yaml
...skipping file, reason unknown...
processing file number:  149  filename:  875525.yaml
...skipping file, reason unknown...
processing file number:  150  filename:  524930.yaml
processing file number:  151  filename:  682901.yaml
processing file number:  152  filename:  548312.yaml
processing file number:  153  filename:  810425.yaml
...skipping Test matches...
processing file number:  154  filename:

processing file number:  272  filename:  1199529.yaml
processing file number:  273  filename:  696759.yaml
processing file number:  274  filename:  660123.yaml
processing file number:  275  filename:  546414.yaml
processing file number:  276  filename:  351688.yaml
processing file number:  277  filename:  392216.yaml
processing file number:  278  filename:  656441.yaml
processing file number:  279  filename:  439152.yaml
...skipping Test matches...
processing file number:  280  filename:  947307.yaml
processing file number:  281  filename:  924629.yaml
processing file number:  282  filename:  1168245.yaml
processing file number:  283  filename:  385750.yaml
processing file number:  284  filename:  1114867.yaml
processing file number:  285  filename:  829729.yaml
processing file number:  286  filename:  573023.yaml
processing file number:  287  filename:  548373.yaml
processing file number:  288  filename:  538071.yaml
processing file number:  289  filename:  416241.yaml
processing file

...skipping file, reason unknown...
processing file number:  414  filename:  1022359.yaml
processing file number:  415  filename:  415284.yaml
processing file number:  416  filename:  930579.yaml
processing file number:  417  filename:  1199513.yaml
processing file number:  418  filename:  913649.yaml
processing file number:  419  filename:  979845.yaml
...skipping file, reason unknown...
processing file number:  420  filename:  966749.yaml
processing file number:  421  filename:  760781.yaml
...skipping Test matches...
processing file number:  422  filename:  476600.yaml
processing file number:  423  filename:  1000855.yaml
...skipping Test matches...
processing file number:  424  filename:  313992.yaml
processing file number:  425  filename:  936157.yaml
processing file number:  426  filename:  251495.yaml
processing file number:  427  filename:  895815.yaml
processing file number:  428  filename:  812785.yaml
...skipping file, reason unknown...
processing file number:  429  filename

checks

In [ ]:
# file = '/Users/z001t72/Documents/Training/cricket/data/all_male/' + '931390.yaml'
# with open(file, 'r') as f:
#     df = pd.io.json.json_normalize(yaml.load(f))

In [ ]:
# df1 = df['innings'].apply(pd.Series).stack()
# df1 = df1.reset_index()
# df1 = df1[df1['level_1'] <= 1] #limiting only to 2 innings. this will exclude innings like super over
# df1.columns = ['nr1','nr2','innings']
# df1 = df1[['innings']]
# df1 = pd.DataFrame(df1.innings.values.tolist())
# df1.columns = ['innings_1','innings_2']
# df1['innings'] = df1.innings_1.combine_first(df1.innings_2)
# df1['innings_number'] = np.where(df1['innings_1'].isna(), '2', '1')
# df1 = df1[['innings','innings_number']]
# df1[['deliveries','team']] = pd.DataFrame(df1.innings.values.tolist(), index = df1.index)[['deliveries','team']]
# df1 = df1.set_index(['innings_number', 'team'])['deliveries'].apply(pd.Series).stack()
# df1 = df1.reset_index()
# df1.columns = ['innings_number','team','notreqd','balls']

# df1['ball'] = df1['balls'].apply(lambda x: list(x.keys())[0])
# df1['batsman'] = df1['balls'].apply(lambda x: list(x.values())[0].get('batsman'))
# df1['non_striker'] = df1['balls'].apply(lambda x: list(x.values())[0].get('non_striker'))
# df1['runs_batsman'] = df1['balls'].apply(lambda x: list(x.values())[0].get('runs', {}).get('batsman'))
# df1['runs_extras'] = df1['balls'].apply(lambda x: list(x.values())[0].get('runs', {}).get('extras'))
# df1['runs_total'] = df1['balls'].apply(lambda x: list(x.values())[0].get('runs', {}).get('total'))
# df1['wicket_kind'] = df1['balls'].apply(lambda x: list(x.values())[0].get('wicket', {}).get('kind'))
# df1['wicket_player_out'] = df1['balls'].apply(lambda x: list(x.values())[0].get('wicket', {}).get('player_out'))
# df1['wicket_fielders'] = df1['balls'].apply(lambda x: list(x.values())[0].get('wicket', {}).get('fielders'))

# # some dates are available as string and some as datetime
# if type(df['info.dates'][0][0]) is str: df1['match_date'] = str(df['info.dates'][0][0])
# elif type(df['info.dates'][0][0]) is datetime.date: df1['match_date'] = "{:%Y-%m-%d}".format(df['info.dates'][0][0])
# else: df1['match_date'] = 'NA'

# df1['match_type'] = df['info.match_type'][0]
# df1['teams'] = str(df['info.teams'][0])
# df1['venue'] = df['info.venue'][0]
# #         df1['city'] = df["info.city"][0]
# df1['overs'] = df['info.overs'][0]
# df1['winner'] = df['info.outcome.winner'][0]

# df1 = df1[columns]